In [1]:
from sklearn.decomposition import PCA
from pennylane import numpy as np
import pennylane as qml
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import combinations
from scipy import stats as st
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

# Function Definitions and Quantum Device Setup

In [151]:
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

In [152]:
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)

In [153]:
def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    np.random.seed(5687)
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

In [198]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

In [156]:
@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

In [157]:
def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [5]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [159]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [160]:
def train(params, X_train, y_train, steps, subset_size, lr,alignment_target=None, idx=None):
    opt = qml.GradientDescentOptimizer(lr)

    for i in range(steps):
        # Choose subset of datapoints to compute the KTA on.
        if idx:
            subset = idx
        else:
            subset = np.random.choice(list(range(len(X_train))), subset_size)
        # Define the cost function for optimization
        cost = lambda _params: -target_alignment(
          X_train[subset],
          y_train[subset],
          lambda x1, x2: kernel(x1, x2, _params),
          assume_normalized_kernel=True,
        )
        # Optimization step
        params = opt.step(cost, params)

        # Report the alignment on the full dataset every 50 steps.
        if (i + 1) % 50 == 0:
            current_alignment = target_alignment(
              X_train,
              y_train,
              lambda x1, x2: kernel(x1, x2, params),
              assume_normalized_kernel=True,
            )
            print(f"Step {i+1} - Alignment = {current_alignment:.3f}")
            if alignment_target and (current_alignment >= alignment_target):
                break
    return params

In [2]:
from sklearn.datasets import load_iris
data = load_iris()

In [3]:
Y = np.copy(data['target'])
X = data['data']
# pca = PCA(n_components=2)
# X = pca.fit_transform(X)
scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)*np.pi

In [6]:
for ker in ['rbf', 'linear', 'poly']:
    acc = []
    for rdm in [5,  8,  9, 15, 17, 30, 31, 34, 41, 42]:
        X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
        clf = SVC(kernel=ker)
        clf.fit(X_train, y_train)
        acc.append(accuracy(clf, X_test, y_test)*100)
    print(f"Kernel: {ker}\t Mean: {np.mean(acc):.2f} \t Std: {np.std(acc):.2f}")

Kernel: rbf	 Mean: 92.67 	 Std: 2.00
Kernel: linear	 Mean: 92.89 	 Std: 0.89
Kernel: poly	 Mean: 92.44 	 Std: 3.01


In [199]:
quantum_accuracies_trained = []
quantum_accuracies_untrained = []
for rdm in [5,  8,  9, 15, 17, 30, 31, 34, 41, 42]:
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    pred_all_trained = []
    pred_all_untrained = []
    all_params = {}
    init_params = random_params(num_wires=3, num_layers=6)
    init_kernel = lambda x1, x2: kernel(x1, x2, init_params)
    for y1, y2 in combinations(np.unique(Y).numpy(), r=2):
        mask = (y_train==y1) | (y_train==y2)
        y_train_pair = y_train[mask]
        X_train_pair = X_train[mask]
        y_train_pair [y_train_pair == y1] = -1
        y_train_pair [y_train_pair == y2] = 1
        svm_untrained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X_train_pair, y_train_pair)
        test_mask = (y_test==y1) | (y_test==y2)
        y_test_pair = y_test[test_mask]
        X_test_pair = X_test[test_mask]
        y_test_pair [y_test_pair == y1] = -1
        y_test_pair [y_test_pair == y2] = 1
        y_pred_pair = svm_untrained.predict(X_test_pair)
        test_accuracy = 1 - np.count_nonzero(y_pred_pair - y_test_pair) / len(y_test_pair)
        print(f"Test Accuracy:{test_accuracy*100:.2f}")
        lr = 2
        new_test_accuracy = test_accuracy
        while (new_test_accuracy != 1) and (new_test_accuracy <= test_accuracy) and (lr >= 0.5):
            if lr >=1 :
                iterations= 500
            else:
                iterations= 1000
            params = all_params.get((y1, y2), init_params)
            params = train(params, X_train_pair, y_train_pair, iterations, 4, lr)
            all_params[(y1, y2)] = params
            trained_kernel = lambda x1, x2: kernel(x1, x2, params)
            svm_trained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)).fit(X_train_pair, y_train_pair)
            y_pred_pair_new = svm_trained.predict(X_test_pair)
            new_test_accuracy = 1 - np.count_nonzero(y_pred_pair_new - y_test_pair) / len(y_test_pair)
            print(f"Test Accuracy:{new_test_accuracy*100:.2f}")
            lr /= 2
        untrained_predictions = svm_untrained.predict(X_test)
        pred_all_untrained.append((untrained_predictions, (y1,y2)))
        if new_test_accuracy <= test_accuracy:
            pred_all_trained.append((untrained_predictions, (y1,y2)))
        else:
            pred_all_trained.append((svm_trained.predict(X_test), (y1,y2)))
    y_pred_all = []
    for y_pred_pair, labels in pred_all_untrained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_untrained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_untrained.append(final_accuracy_untrained*100)
    print(f"Final Accuracy Untrained {final_accuracy_untrained*100:.2f}")
    y_pred_all = []
    for y_pred_pair, labels in pred_all_trained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_trained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_trained.append(final_accuracy_trained*100)
    print(f"Final Accuracy Trained {final_accuracy_trained*100:.2f}")

Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Final Accuracy Untrained 100.00
Final Accuracy Trained 100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:93.33
Step 50 - Alignment = 0.446
Step 100 - Alignment = 0.452
Step 150 - Alignment = 0.472
Step 200 - Alignment = 0.458
Step 250 - Alignment = 0.472
Step 300 - Alignment = 0.439
Step 350 - Alignment = 0.437
Step 400 - Alignment = 0.456
Step 450 - Alignment = 0.430
Step 500 - Alignment = 0.460
Test Accuracy:96.67
Final Accuracy Untrained 95.56
Final Accuracy Trained 97.78
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:96.67
Step 50 - Alignment = 0.428
Step 100 - Alignment = 0.431
Step 150 - Alignment = 0.414
Step 200 - Alignment = 0.424
Step 250 - Alignment = 0.449
Step 300 - Alignment = 0.447
Step 350 - Alignment = 0.444
Step 400 - Alignment = 0.448
Step 450 - Alignment = 0.462
Step 500 - Alignment = 0.456
Test Accuracy:96.67
Step 50 - Alignment = 0.458
Step 100 - Alignment = 0.452
Step 150 - Align

In [190]:
np.mean(quantum_accuracies_trained)

94.44444444444446

In [191]:
np.std(quantum_accuracies_trained)

2.048787657176196

In [192]:
np.mean(quantum_accuracies_untrained)

92.0

In [193]:
np.std(quantum_accuracies_untrained)

2.0367003088692623